# Building an Anime Recommender System

In [1]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 1. Load Dataset and Model

In [2]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 3. Webscrape current profile and create prediction

In [3]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 166 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7


In [4]:
rhat = model.predict(x)

In [5]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [6]:
prediction.sort_values(by='score', ascending=False).head(10)

,score,title,title_english,type,premiered,genre
11757,9.670687,Sword Art Online,Sword Art Online,TV,Summer 2012,"Action, Adventure, Fantasy, Game, Romance"
530,9.578186,Bishoujo Senshi Sailor Moon,Sailor Moon,TV,Spring 1992,"Demons, Magic, Romance, Shoujo"
532,9.350300,Bishoujo Senshi Sailor Moon S,Sailor Moon S,TV,Spring 1994,"Drama, Magic, Romance, Shoujo"
740,9.251397,Bishoujo Senshi Sailor Moon R,Sailor Moon R,TV,Spring 1993,"Demons, Magic, Romance, Shoujo"
996,8.987871,Bishoujo Senshi Sailor Moon: Sailor Stars,Sailor Moon Sailor Stars,TV,Spring 1996,"Adventure, Comedy, Drama, Fantasy, Magic, Roma..."
5690,8.934628,Nodame Cantabile Finale,NaN,TV,Winter 2010,"Comedy, Josei, Music, Romance"
13125,8.928691,Shinsekai yori,From the New World,TV,Fall 2012,"Sci-Fi, Mystery, Horror, Psychological, Supern..."
21881,8.880716,Sword Art Online II,Sword Art Online II,TV,Summer 2014,"Action, Game, Adventure, Romance, Fantasy"
4477,8.853769,Nodame Cantabile: Paris-hen,NaN,TV,Fall 2008,"Music, Slice of Life, Comedy, Romance, Josei"
11981,8.840688,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,Puella Magi Madoka Magica the Movie: Rebellion,Movie,NaN,"Psychological, Drama, Magic, Thriller"
